# Import Modules

In [22]:
import os
import sys

import pickle

import pandas as pd

# Read Data

In [2]:
file_path = "/home/raulf2012/Dropbox/01_norskov/00_git_repos/PROJ_IrOx_Active_Learning_OER/workflow/ml_modelling/00_ml_workflow/190611_new_workflow/02_gaus_proc/out_data/temp.pickle"

with open(file_path, "rb") as fle:
    data_dict = pickle.load(fle)

df_bulk_dft = data_dict["df_bulk_dft"]
model_i = data_dict["model_i"]

In [3]:
print("df_bulk_dft.shape:", df_bulk_dft.shape)

df_bulk_dft.head()

df_bulk_dft.shape: (208, 9)


,atoms,energy_pa,form_e_chris,id,id_old,path,source,stoich,energy
id_unique,,,,,,,,,
61mwz4ny85,"(Atom('Ir', [0.0, 0.0, 0.0], index=0), Atom('O...",-4.702486,NaN,NaN,124,/global/cscratch1/sd/flores12/IrOx_Project_tem...,raul,AB3,-18.809946
62xp6e9yzs,"(Atom('Ir', [-2.81342, 14.60929, 5.52693], ind...",-6.226849,NaN,NaN,230,/global/cscratch1/sd/flores12/IrOx_Project_tem...,raul,AB3,-298.888752
6384vt7pml,"(Atom('O', [2.24653, 5.50212, 1.81723], index=...",-6.397915,NaN,NaN,237,/global/cscratch1/sd/flores12/IrOx_Project_tem...,raul,AB3,-358.283250
63xlmhno8h,"(Atom('O', [0.37755, 1.807, 0.32305], index=0)...",-6.184285,NaN,NaN,131,/global/cscratch1/sd/flores12/IrOx_Project_tem...,raul,AB3,-49.474282
64vo6w8wvq,"(Atom('O', [12.74778, 1.5778, 0.20169], index=...",-6.138180,NaN,NaN,91,/global/cscratch1/sd/flores12/IrOx_Project_tem...,raul,AB3,-196.421749


In [4]:
print("model_i.shape:", model_i.shape)

model_i.head()

model_i.shape: (208, 15)


,prediction,uncertainty,uncertainty_with_reg,id,computed,prediction_unstandardized,uncertainty_unstandardized,uncertainty_with_reg_unstandardized,energy_pa,x_axis_ind,actually_computed,tmp,marker_size,marker_line_color,marker_line_size
id_unique,,,,,,,,,,,,,,,
61mwz4ny85,3.224219,0.000058,0.000115,124.0,True,-4.702486,0.000024,0.000048,-4.702486,197,True,-4.702511,10,red,1.5
62xp6e9yzs,-0.413439,0.000058,0.000115,230.0,True,-6.226849,0.000024,0.000048,-6.226849,42,True,-6.226873,10,red,1.5
6384vt7pml,-0.821662,0.000058,0.000115,237.0,True,-6.397915,0.000024,0.000048,-6.397915,14,True,-6.397939,10,red,1.5
63xlmhno8h,-0.311867,0.000058,0.000115,131.0,True,-6.184285,0.000024,0.000048,-6.184285,45,True,-6.184309,10,red,1.5
64vo6w8wvq,-0.201843,0.000058,0.000115,91.0,True,-6.138180,0.000024,0.000048,-6.138180,56,True,-6.138204,10,red,1.5


In [5]:
df_tmp = pd.concat(
    [
        model_i[["x_axis_ind", "computed", "actually_computed"]],
        df_bulk_dft["energy_pa"],
        ],
    axis=1, sort=True,
    )
print("df_tmp.shape:", df_tmp.shape)

df_tmp = df_tmp.reindex(model_i.index.sort_values().tolist())

df_tmp.shape: (208, 2)


In [6]:
df_tmp["energy_pa"].isnull()

def method(row_i):
#     energy_pa = row_i["energy_pa"]
    if pd.isna(row_i["energy_pa"]): size=5
    else: size=10
    return(size)

df_i = df_tmp
df_i["marker_size"] = df_i.apply(method, axis=1)
df_tmp = df_i

In [7]:
# df_tmp.fillna(value=-5.)

In [8]:
df_tmp[df_tmp["energy_pa"].isna()].shape

(101, 3)

In [9]:
print(df_tmp[df_tmp["energy_pa"].isna()].shape)

print(df_tmp[~df_tmp["energy_pa"].isna()].shape)

(101, 3)
(107, 3)


# Getting IDs of Next Systems to Run
---

In [1]:
import time
import os

import pickle

import pandas as pd

import chart_studio.plotly as py
from chart_studio.grid_objs import Grid, Column
import plotly.graph_objects as go

from plotting.my_plotly import my_plotly_plot

from methods import (
    get_sliders_init_dict,
    get_slider_step_i,
    get_layout,
    )

# stoich_i = "AB2"
stoich_i = "AB2"


# duration_long = 1000 * 2
# duration_short = 800 * 2

duration_long = 1000 * 3
duration_short = 800 * 5

save_plot = False
if save_plot:
    save_dir = "04_IrOx_surfaces_OER" 
else:
    save_dir = "__temp__/PROJ_irox/190625"

layout = get_layout(
    duration_long=duration_long,
    duration_short=duration_short)

sliders_dict = get_sliders_init_dict(duration_short)

In [45]:
al_output_data_path = os.path.join(
    os.environ["PROJ_irox"],
    "workflow/ml_modelling", "00_ml_workflow/190611_new_workflow",
    "02_gaus_proc/out_data",
    "data_dict_" + stoich_i + ".pickle",
    # "data_dict_AB3.pickle",
    )

with open(al_output_data_path, "rb") as fle:
    al_data_dict = pickle.load(fle)

model_i = al_data_dict[43]["model"].sort_values("prediction_unstandardized")

model_i = model_i[model_i["actually_computed"] == False]

nu94n3v4n4    508.0
crnonhbf6j     10.0
zoxsbkmyna    649.0
7ynav474cs    543.0
vtzi7tcony    662.0
              ...  
7ymj7t9l8p    531.0
nk8en4m1va    346.0
cfzam1mdbf    590.0
9d6y74b5by    633.0
mwxdzp8g7u    642.0
Name: id, Length: 342, dtype: float64

In [46]:
rootdir = os.path.join(
    os.environ["dropbox"],
    "__temp__")

with open(os.path.join(rootdir, "ids_in_job_folders_iro2_nersc.txt"), 'r') as f:
    x = f.readlines()
    iro2_nersc_ids = [i.strip() for i in x]

with open(os.path.join(rootdir, "ids_in_job_folders_iro2_sher.txt"), 'r') as f:
    x = f.readlines()
    iro2_sher_ids = [i.strip() for i in x]

In [47]:
print(len(iro2_nersc_ids))
print(len(iro2_sher_ids))

172
172


In [48]:
iro2_ids = list(set(iro2_nersc_ids + iro2_sher_ids))

print(len(iro2_ids))

327


In [71]:
"508" in iro2_ids

False

In [64]:
# [i for i in str(int(model_i["id"].iloc[0])).zfill(3) if i not in iro2_ids]


# str(int(model_i["id"].iloc[0])).zfill(3)

ids_to_run = []
for i in model_i["id"]:
    tmp = 42

    id_str = str(int(i)).zfill(3)
    
    if id_str not in iro2_ids:
        ids_to_run.append(id_str)

In [75]:
[int(i) for i in ids_to_run]

[508,
 649,
 543,
 662,
 591,
 461,
 424,
 36,
 54,
 43,
 374,
 4,
 679,
 524,
 551,
 683,
 483,
 544,
 429,
 579,
 335,
 479,
 522,
 445,
 465,
 357,
 597,
 538,
 93,
 402,
 566,
 493,
 418,
 386,
 384,
 604,
 367,
 562,
 599,
 526,
 671,
 63,
 32,
 331,
 525,
 462,
 559,
 670,
 455,
 614,
 669,
 362,
 94,
 589,
 441,
 365,
 372,
 572,
 95,
 438,
 96,
 394,
 6,
 399,
 678,
 354,
 509,
 37,
 66,
 97,
 67,
 690,
 623,
 453,
 624,
 663,
 696,
 393,
 449,
 500,
 584,
 608,
 494,
 505,
 638,
 82,
 539,
 602,
 350,
 640,
 486,
 692,
 358,
 84,
 61,
 410,
 609,
 643,
 580,
 46,
 636,
 569,
 639,
 373,
 90,
 625,
 495,
 472,
 504,
 587,
 492,
 605,
 55,
 408,
 40,
 528,
 619,
 645,
 428,
 529,
 369,
 446,
 33,
 666,
 657,
 360,
 56,
 333,
 468,
 688,
 481,
 607,
 89,
 341,
 404,
 595,
 42,
 405,
 389,
 659,
 391,
 617,
 60,
 613,
 466,
 421,
 694,
 351,
 561,
 549,
 610,
 379,
 443,
 44,
 49,
 52,
 618,
 665,
 420,
 650,
 91,
 489,
 457,
 533,
 329,
 680,
 343,
 631,
 400,
 8,
 684,
 516,
 43

In [73]:
259 + 327

586

In [21]:
# sys.path.insert(0, rootdir)
# from ids_to_run_nersc import ids_to_run as ids_to_run_nersc
# from ids_to_run_sher import ids_to_run as ids_to_run_sher
# ids_already_running = ids_to_run_nersc + ids_to_run_sher
# ids_already_running = list(set(ids_already_running))
# ids_already_running